# Quantum Library Explained

This notebook explains how the quantum decision-making system works.

## Overview

The quantum library provides two methods for making tactical decisions:
1. **Exact Diagonalization**: Fast, finds exact ground state
2. **QAOA**: Realistic quantum algorithm, slower but more representative of real quantum hardware

In [ ]:
import sys
sys.path.append('..')

import quantum_library as qlib
import numpy as np

## Configuration: Choosing the Quantum Method

You can easily switch between methods:

In [ ]:
# Check current method
print(f"Current method: {qlib.QUANTUM_METHOD}")

# Switch to exact (fast)
qlib.set_quantum_method("exact")

# Or switch to QAOA (realistic)
# qlib.set_quantum_method("qaoa")
# qlib.QAOA_P = 2
# qlib.QAOA_N_RESTARTS = 3

## How the H-value System Works

The quantum system uses an H-value to evaluate positions:
- **Lower H** = More offensive/advantageous position
- **Higher H** = More defensive/vulnerable position

### H Calculation

```python
H = offensive_power - vulnerability
```

With team size consideration:
- **If outnumbered**: `H = offensive - (num_diff) * vulnerability` (defensive)
- **If advantage**: `H = (num_diff) * offensive - vulnerability` (offensive)

In [ ]:
# Example: Calculate H gradients
from battlefield import Soldier

# Create a test soldier
soldier = Soldier(x=1, y=1, strength=2, health=2, unit_type='soldier', team='Quantum', range_dist=1)

# Create some enemies
enemies = [
    Soldier(x=2, y=1, strength=2, health=2, unit_type='soldier', team='Classical', range_dist=1),
    Soldier(x=2, y=2, strength=2, health=2, unit_type='soldier', team='Classical', range_dist=1),
]

teammates = [
    Soldier(x=0, y=1, strength=2, health=2, unit_type='soldier', team='Quantum', range_dist=1),
]

# Calculate H gradients in all 4 directions
h_left = qlib.calculate_h_left(soldier, enemies, teammates)
h_right = qlib.calculate_h_right(soldier, enemies, teammates)
h_up = qlib.calculate_h_up(soldier, enemies, teammates)
h_down = qlib.calculate_h_down(soldier, enemies, teammates)

print("H Gradients:")
print(f"  Left: {h_left}")
print(f"  Right: {h_right}")
print(f"  Up: {h_up}")
print(f"  Down: {h_down}")
print("\nNegative gradient = better direction (more offensive)")

## Quantum Step: Making a Decision

The quantum step function takes H gradients and finds the optimal move.

In [ ]:
# Example omega dict (from H gradients)
omega = {
    "I": int(h_left),   # Left
    "D": int(h_right),  # Right
    "+": int(h_up),     # Up
    "-": int(h_down)    # Down
}

print(f"Omega values: {omega}")

# Make a quantum decision
current_x, current_y = 1, 1
new_x, new_y = qlib.quantum_step(current_x, current_y, omega)

print(f"\nDecision: ({current_x}, {current_y}) → ({new_x}, {new_y})")
print(f"Movement: dx={new_x-current_x}, dy={new_y-current_y}")

## EXACT vs QAOA Comparison

Let's compare both methods on the same problem:

In [ ]:
import time

test_omega = {"I": 1, "D": -2, "+": 0, "-": -1}
start_pos = (2, 2)

# Test EXACT method
qlib.set_quantum_method("exact")
start = time.time()
result_exact = qlib.quantum_step(start_pos[0], start_pos[1], test_omega)
time_exact = time.time() - start

# Test QAOA method (if you want, may take longer)
# qlib.set_quantum_method("qaoa")
# qlib.QAOA_P = 1
# qlib.QAOA_N_RESTARTS = 2
# start = time.time()
# result_qaoa = qlib.quantum_step(start_pos[0], start_pos[1], test_omega)
# time_qaoa = time.time() - start

print(f"EXACT method:")
print(f"  Result: {start_pos} → {result_exact}")
print(f"  Time: {time_exact*1000:.2f} ms")

# print(f"\nQAOA method:")
# print(f"  Result: {start_pos} → {result_qaoa}")
# print(f"  Time: {time_qaoa:.2f} s")

## QAOA Internals

When using QAOA, the system:
1. Builds a quantum Hamiltonian from omega values
2. Creates a QAOA circuit with p layers
3. Optimizes circuit parameters (gammas, betas) using scipy
4. Extracts the most likely bitstring
5. Caches the result for future reuse

In [ ]:
# Example: Run QAOA with detailed output
qlib.set_quantum_method("qaoa")
qlib.QAOA_P = 1  # Use p=1 for speed
qlib.QAOA_N_RESTARTS = 2

omega_test = {"I": 2, "D": -3, "+": 1, "-": 0}

# Run QAOA optimization directly to see details
best, idx = qlib.qaoa_optimize_no_xx(
    omega=omega_test,
    p=1,
    n_restarts=2,
    seed=42
)

print("QAOA Optimization Result:")
print(f"  Energy: {best['E']:.4f}")
print(f"  Bitstring: |{best['bit']}>")
print(f"  Probability: {best['pbit']:.4f}")
print(f"  Optimal angles: {best['theta'][:2]} (gammas), {best['theta'][2:]} (betas)")

## Cache Management

QAOA results are cached to avoid recomputation:

In [ ]:
# Check cache size
print(f"QAOA cache entries: {len(qlib._qaoa_cache)}")

# Clear cache if needed
qlib.clear_qaoa_cache()
print(f"After clearing: {len(qlib._qaoa_cache)}")

## Summary

**Key Functions**:
- `calculate_h_value()`: Compute H heuristic
- `calculate_h_left/right/up/down()`: Compute H gradients
- `quantum_step()`: Main decision function (dispatches to exact or QAOA)
- `quantum_step_exact()`: Fast exact diagonalization
- `quantum_step_qaoa()`: QAOA variational algorithm

**Configuration**:
- `QUANTUM_METHOD`: "exact" or "qaoa"
- `QAOA_P`: Circuit depth (1-3 recommended)
- `QAOA_N_RESTARTS`: Optimizer restarts (2-5 recommended)

**Best Practices**:
- Use EXACT for testing and rapid iteration
- Use QAOA for research and realistic quantum simulation
- Start with low QAOA parameters and increase as needed
- Clear cache when changing QAOA parameters